In [24]:
from icecream import ic
import pandas as pd
import numpy as np
import os
import re

basepath = os.getcwd()
raw_txt_dir = "data"
full_path = os.path.join(basepath, raw_txt_dir)

def parsefile(file):
    try:
        with open(file, mode="r") as f:
            contents = f.read()
            return contents
    except Exception as e:
        print(e)    
        
        
def tab_regex(stream):
    tab_parts = re.split(r'\[t\]', stream)
    assert(len(tab_parts)>1)
    
for folder in os.listdir(full_path):
    try:
        for file in os.listdir(os.path.join(full_path, folder)):
            if str(file) != "Readme.txt" and str(file) != ".DS_Store":
                raw_text = parsefile(os.path.join(full_path, folder, file))    
    except NotADirectoryError:
        pass
    
    

norton.txt
[t]
softwa
Nokia 6600.txt
[t]
phone[
Hitachi router.txt
[t]
perfor
ipod.txt
##Don't ge
Diaper Champ.txt
[t]
Works[
Linksys Router.txt
[t]
router
Canon S100.txt
[t]
small[
MicroMP3.txt
[t]
##The 
Canon PowerShot SD500.txt
'utf-8' codec can't decode byte 0x96 in position 1748: invalid start byte
Computer.txt
## I purch
Speaker.txt
speakers[+
Router.txt
price[+1],
Nokia 6610.txt
**********
Canon G3.txt
**********
Creative Labs Nomad Jukebox Zen Xtra 40GB.txt
**********
Apex AD2600 Progressive-scan DVD player.txt
**********
Nikon coolpix 4300.txt
**********
